<a href="https://colab.research.google.com/github/lucasaxm/desafio_ia_alura_google/blob/main/Desafio_Imers%C3%A3o_IA_Alura_%2B_Google.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Esse notebook irá buscar pelo user definido abaixo no site letterboxd e com base nos seus reviews irá gerar 5 recomendações de filmes no idioma definido abaixo.

In [2]:
letterboxd_username = 'umaboatarde'
language = 'Brazilian Portuguese'
sample_size = 15

Letterboxd scraper baseado nesse [artigo](https://medium.com/@alf.19x/letterboxd-friends-ranker-simple-movie-recommendation-system-80a38dcfb0da).

In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import textwrap
from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

def get_random_rows(df, n):
  """Returns n random rows from a DataFrame, or the whole DataFrame if it has fewer than n rows.

  Args:
    df: The DataFrame to sample from.
    n: The number of rows to sample.

  Returns:
    A DataFrame containing n random rows from df, or the whole df if it has fewer than n rows.
  """

  return df.sample(n=min(n, len(df)))

class LetterboxdScraper:
    def __init__(self):
        self.DOMAIN = "https://letterboxd.com"

    def transform_ratings(self, some_str):
        """
        Transforms raw star rating into float value.
        :param: some_str: actual star rating
        :rtype: returns the float representation of the given star(s)
        """
        stars = {
            "★": 1,
            "★★": 2,
            "★★★": 3,
            "★★★★": 4,
            "★★★★★": 5,
            "½": 0.5,
            "★½": 1.5,
            "★★½": 2.5,
            "★★★½": 3.5,
            "★★★★½": 4.5
        }
        try:
            return stars[some_str]
        except:
            return -1

    def scrape_films(self, username):
        """
        Scrapes film data for the given username.
        :param: username: Letterboxd username to scrape
        :rtype: pandas DataFrame containing film data
        """
        movies_dict = {}
        movies_dict['id'] = []
        movies_dict['title'] = []
        movies_dict['rating'] = []
        movies_dict['liked'] = []
        movies_dict['link'] = []
        url = self.DOMAIN + "/" + username + "/films/"
        url_page = requests.get(url)
        soup = BeautifulSoup(url_page.content, 'html.parser')

        # check number of pages
        li_pagination = soup.findAll("li", {"class": "paginate-page"})
        if len(li_pagination) == 0:
            ul = soup.find("ul", {"class": "poster-list"})
            if (ul != None):
                movies = ul.find_all("li")
                for movie in movies:
                    movies_dict['id'].append(movie.find('div')['data-film-id'])
                    movies_dict['title'].append(movie.find('img')['alt'])
                    movies_dict['rating'].append(self.transform_ratings(movie.find('p', {"class": "poster-viewingdata"}).get_text().strip()))
                    movies_dict['liked'].append(movie.find('span', {'class': 'like'})!=None)
                    movies_dict['link'].append(movie.find('div')['data-target-link'])
        else:
            for i in range(int(li_pagination[-1].find('a').get_text().strip())):
                url = self.DOMAIN + "/" + username + "/films/page/" + str(i+1)
                url_page = requests.get(url)
                soup = BeautifulSoup(url_page.content, 'html.parser')
                ul = soup.find("ul", {"class": "poster-list"})
                if (ul != None):
                    movies = ul.find_all("li")
                    for movie in movies:
                        movies_dict['id'].append(movie.find('div')['data-film-id'])
                        movies_dict['title'].append(movie.find('img')['alt'])
                        movies_dict['rating'].append(self.transform_ratings(movie.find('p', {"class": "poster-viewingdata"}).get_text().strip()))
                        movies_dict['liked'].append(movie.find('span', {'class': 'like'})!=None)
                        movies_dict['link'].append(movie.find('div')['data-target-link'])

        df_film = pd.DataFrame(movies_dict)
        return df_film

Configuração do Gemini e prompt usado para gerar as recomendações.

In [4]:
import google.generativeai as genai
# Used to securely store your API key
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

generation_config = {
  "candidate_count": 1,
  "temperature": 0.5,
}

safety_settings={
  'HATE': 'BLOCK_NONE',
  'HARASSMENT': 'BLOCK_NONE',
  'SEXUAL' : 'BLOCK_NONE',
  'DANGEROUS' : 'BLOCK_NONE'
}

model = genai.GenerativeModel(model_name = "gemini-1.0-pro-latest",
                              generation_config=generation_config,
                              safety_settings=safety_settings)
def generate_prompt(df, language):
  return ("Based on the film ratings table below, recommend 5 movies I might enjoy that are not already listed in the table.\n"
          "IMPORTANT: 0 is the lowest rating and 5 is the highest rating, don't recommend movies similar to those that are scored below 2.5.\n"
          "For each recommendation, provide a brief sinopsys and explain why you think I'll enjoy it based on my past ratings. "
          f"Your response should be formated as markdown cards with the Title in {language}, and below that the description on why you recommend this movie.\n"
          f"{df[['title','rating']].to_csv(index=False)}")

In [5]:
scraper = LetterboxdScraper()
df_letterboxd = scraper.scrape_films(letterboxd_username)
df_letterboxd_sample = df_letterboxd.sample(n=min(sample_size, len(df_letterboxd)))
df_letterboxd_sample

,id,title,rating,liked,link
24,49062,Howl's Moving Castle,5.0,True,/film/howls-moving-castle/
21,187986,Ex Machina,4.5,False,/film/ex-machina-2015/
23,51365,Mr. & Mrs. Smith,4.0,False,/film/mr-mrs-smith-2005/
8,996925,BEEF,4.5,True,/film/beef-2023/
1,617443,Dune: Part Two,4.5,True,/film/dune-part-two/
9,995847,Waco: American Apocalypse,3.5,False,/film/waco-american-apocalypse/
13,449442,Bodies Bodies Bodies,3.5,False,/film/bodies-bodies-bodies/
5,755564,The Holdovers,4.0,True,/film/the-holdovers/
25,51936,Pretty Woman,4.0,False,/film/pretty-woman/
15,507902,Words Bubble Up Like Soda Pop,4.0,False,/film/words-bubble-up-like-soda-pop/


In [6]:
prompt = generate_prompt(df_letterboxd_sample, language)
print(prompt)

Based on the film ratings table below, recommend 5 movies I might enjoy that are not already listed in the table.
IMPORTANT: 0 is the lowest rating and 5 is the highest rating, don't recommend movies similar to those that are scored below 2.5.
For each recommendation, provide a brief sinopsys and explain why you think I'll enjoy it based on my past ratings. Your response should be formated as markdown cards with the Title in Brazilian Portuguese, and below that the description on why you recommend this movie.
title,rating
Howl's Moving Castle,5.0
Ex Machina,4.5
Mr. & Mrs. Smith,4.0
BEEF,4.5
Dune: Part Two,4.5
Waco: American Apocalypse,3.5
Bodies Bodies Bodies,3.5
The Holdovers,4.0
Pretty Woman,4.0
Words Bubble Up Like Soda Pop,4.0
Mad Max: Fury Road,4.0
The Proposal,4.0
Suzume,3.0
mother!,3.5
Past Lives,4.0



Execução do prompt e display em markdown



In [7]:
response = model.generate_content(prompt)

display(to_markdown(response.text))

> ### A Bruxa (The Witch)
> 
> Este filme de terror sobrenatural ambientado na Nova Inglaterra do século XVII explora temas de religião, superstição e a natureza do mal. Acredito que você apreciará a atmosfera sombria e inquietante do filme, bem como sua exploração complexa dos personagens.
> 
> ### O Farol (The Lighthouse)
> 
> Este drama psicológico em preto e branco conta a história de dois faroleiros isolados em uma ilha remota. O filme é visualmente deslumbrante e apresenta performances hipnotizantes de Robert Pattinson e Willem Dafoe. Acredito que você apreciará a atmosfera claustrofóbica e o comentário do filme sobre isolamento e loucura.
> 
> ### Suspiria (2018)
> 
> Este remake do clássico filme de terror italiano é uma obra-prima visualmente deslumbrante e assustadora. O filme segue uma jovem bailarina que se matricula em uma prestigiosa academia de dança, apenas para descobrir que ela é um lar para um culto de bruxas. Acredito que você apreciará a atmosfera surreal e perturbadora do filme, bem como sua exploração dos temas de feminilidade e trauma.
> 
> ### A Chegada (Arrival)
> 
> Este filme de ficção científica conta a história de uma linguista encarregada de se comunicar com alienígenas que chegaram à Terra. O filme é inteligente, comovente e visualmente deslumbrante. Acredito que você apreciará a exploração do filme sobre temas de linguagem, comunicação e a natureza da humanidade.
> 
> ### O Babadook (The Babadook)
> 
> Este filme de terror psicológico australiano é uma exploração assustadora e comovente da maternidade, luto e doença mental. O filme apresenta uma performance incrível de Essie Davis como uma mãe solteira que luta para criar seu filho problemático. Acredito que você apreciará a atmosfera arrepiante do filme e sua exploração complexa dos personagens.